In [8]:
import geopandas as gpd
import networkx as nx
import folium

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Iterate through the GeoDataFrame
for index, row in gdf.iterrows():
    if 'power' in row and row['power'] == 'tower':
        point = row['geometry']
        coordinates = (point.y, point.x)  # Swap latitude and longitude
        node_id = row['id']
        G.add_node(node_id, pos=coordinates)  # Store coordinates as node attribute

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add markers for power tower nodes
for node_id, attrs in G.nodes(data=True):
    coords = attrs['pos']
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Save the map as an HTML file
m.save('power_tower_network.html')

# Display the map in the notebook
m

In [5]:
import geopandas as gpd
import networkx as nx
import folium

from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    G.add_edge(tower_nodes[i][0], tower_nodes[j][0], weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Update the folium map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links.html')


KeyError: 'pos'

In [3]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Verify all nodes have 'pos' attribute
for node_id, attrs in G.nodes(data=True):
    if 'pos' not in attrs:
        print(f"Node {node_id} is missing 'pos' attribute!")

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph and verify nodes exist
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    
    if node_i not in G:
        print(f"Node {node_i} not in graph!")
    if node_j not in G:
        print(f"Node {node_j} not in graph!")
    
    G.add_edge(node_i, node_j, weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add edges to the map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m.save('power_tower_network_with_links.html')


In [4]:
import json
import pandas as pd

# Load the existing GeoJSON file
with open('export.geojson', 'r') as file:
    geojson = json.load(file)

# Read the CSV file
df = pd.read_csv('ManualPowerPoles.csv', header=None)

# Convert CSV data to a list of coordinates
new_coords = [list(map(float, coord.split(','))) for coord in df[0]]

# Calculate the next available node id based on the last one
last_node_id = int(geojson['features'][-1]['properties']['@id'].split('/')[-1])
next_node_id = last_node_id + 1

# Add new nodes to the GeoJSON structure
for coord in new_coords:
    feature = {
        "type": "Feature",
        "properties": {
            "@id": f"node/{next_node_id}",
            "power": "tower"
        },
        "geometry": {
            "type": "Point",
            "coordinates": [coord[1], coord[0]]  # Note the order: [longitude, latitude]
        },
        "id": f"node/{next_node_id}"
    }
    
    geojson['features'].append(feature)
    next_node_id += 1

# Save the updated GeoJSON back to the file
with open('updated_export.geojson', 'w') as file:
    json.dump(geojson, file)


In [23]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Verify all nodes have 'pos' attribute
for node_id, attrs in G.nodes(data=True):
    if 'pos' not in attrs:
        print(f"Node {node_id} is missing 'pos' attribute!")

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph and verify nodes exist
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    
    if node_i not in G:
        print(f"Node {node_i} not in graph!")
    if node_j not in G:
        print(f"Node {node_j} not in graph!")
    
    G.add_edge(node_i, node_j, weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add edges to the map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
m


In [6]:
import geopandas as gpd
import networkx as nx
import folium
import numpy as np
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Function to connect using k-nearest neighbors
def knn_connections(dist_matrix, k=2):
    edges = []
    for i, distances in enumerate(dist_matrix):
        nearest_indices = np.argsort(distances)[1:k+1]  # Exclude the first one since it's the node itself
        for j in nearest_indices:
            edges.append((i, j))
    return edges

# Function to connect based on distance threshold
def threshold_connections(dist_matrix, threshold=1.5):
    edges = []
    for i in range(len(dist_matrix)):
        for j in range(i+1, len(dist_matrix)):
            if dist_matrix[i][j] < threshold:
                edges.append((i, j))
    return edges

# Function to randomly connect nodes
def random_connections(num_nodes, num_edges=10):
    edges = set()
    while len(edges) < num_edges:
        i, j = np.random.choice(num_nodes, 2, replace=False)
        if i > j:
            i, j = j, i
        edges.add((i, j))
    return list(edges)

# List of distinct colors for the different configurations
colors = ['blue', 'green', 'red', 'purple']

configurations = [
    ("Minimum Spanning Tree", [(i, j) for i, j, _ in zip(mst.row, mst.col, mst.data)]),
    ("K-Nearest Neighbors", knn_connections(dist_matrix, k=2)),
    ("Threshold-based Connection", threshold_connections(dist_matrix, threshold=1.5)),
    ("Random Connections", random_connections(len(positions), num_edges=10))
]

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

for color, (config_name, edges) in zip(colors, configurations):
    for i, j in edges:
        locs = [tower_nodes[i][1], tower_nodes[j][1]]
        folium.PolyLine(locs, color=color, weight=2.5, opacity=1, tooltip=config_name).add_to(m)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
new_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

# Add new nodes to the map
for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

# Save the updated map
#m.save('power_tower_network_with_links_updated.html')
m

In [21]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

positions = [coords for _, coords in tower_nodes]
dist_matrix = distance_matrix(positions, positions)
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Function to ensure that each node has a maximum of two connections
def add_edge(connections, i, j):
    if connections.get(i, 0) < 2 and connections.get(j, 0) < 2:
        connections[i] = connections.get(i, 0) + 1
        connections[j] = connections.get(j, 0) + 1
        return True
    return False

connections_count = {}

# MST Connections
mst_edges = [(i, j) for i, j, _ in zip(mst.row, mst.col, mst.data) if add_edge(connections_count, i, j)]

# KNN Connections
k = 2
knn_edges = []
for i, distances in enumerate(dist_matrix):
    nearest_indices = np.argsort(distances)[1:k+1]  # Exclude the first one since it's the node itself
    for j in nearest_indices:
        if add_edge(connections_count, i, j):
            knn_edges.append((i, j))

# Threshold-based Connection
threshold = 2.5
threshold_edges = []
for i in range(len(dist_matrix)):
    for j in range(i+1, len(dist_matrix)):
        if dist_matrix[i][j] < threshold and add_edge(connections_count, i, j):
            threshold_edges.append((i, j))

# Map creation
colors = ['blue', 'green', 'red']
configurations = [
    ("Minimum Spanning Tree", mst_edges),
    ("K-Nearest Neighbors", knn_edges),
    ("Threshold-based Connection", threshold_edges)
]

mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add nodes and edges to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

for color, (config_name, edges) in zip(colors, configurations):
    for i, j in edges:
        locs = [tower_nodes[i][1], tower_nodes[j][1]]
        folium.PolyLine(locs, color=color, weight=2.5, opacity=1, tooltip=config_name).add_to(m)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

new_nodes = []
for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

m


In [22]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

positions = [coords for _, coords in tower_nodes]
dist_matrix = distance_matrix(positions, positions)
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Initialize the graph with MST connections
G = nx.Graph()
for i, j in zip(mst.row, mst.col):
    G.add_edge(i, j)

# Function to ensure each node has a maximum of three connections
def add_edge_if_possible(graph, i, j):
    if graph.degree(i) < 3 and graph.degree(j) < 3:
        graph.add_edge(i, j)
        return True
    return False

# KNN Connections
k = 3
for i, distances in enumerate(dist_matrix):
    nearest_indices = np.argsort(distances)[1:k+1]
    for j in nearest_indices:
        add_edge_if_possible(G, i, j)

# Threshold-based Connection
threshold = 2.5
for i in range(len(dist_matrix)):
    for j in range(i+1, len(dist_matrix)):
        if dist_matrix[i][j] < threshold:
            add_edge_if_possible(G, i, j)

# Connect any remaining isolated nodes/subgraphs
while not nx.is_connected(G):
    min_dist = float('inf')
    closest_pair = None

    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            if G.degree(i) < 3 and G.degree(j) < 3 and not G.has_edge(i, j):
                if dist_matrix[i][j] < min_dist:
                    min_dist = dist_matrix[i][j]
                    closest_pair = (i, j)

    if closest_pair:
        G.add_edge(*closest_pair)
    else:
        break  # Exit if no more edges can be added

# Extract edges
mst_edges = [(i, j) for i, j in G.edges() if (i, j) in zip(mst.row, mst.col) or (j, i) in zip(mst.row, mst.col)]
remaining_edges = [(i, j) for i, j in G.edges() if (i, j) not in mst_edges]

# Map creation
colors = ['blue', 'green']
configurations = [
    ("Minimum Spanning Tree", mst_edges),
    ("Additional Connections", remaining_edges)
]

mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add nodes and edges to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

for color, (config_name, edges) in zip(colors, configurations):
    for i, j in edges:
        locs = [tower_nodes[i][1], tower_nodes[j][1]]
        folium.PolyLine(locs, color=color, weight=2.5, opacity=1, tooltip=config_name).add_to(m)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

new_nodes = []
for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        new_nodes.append(centroid)

for node in new_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

m


In [25]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Introduce additional nodes for large gaps in the MST
introduce_threshold = 2.5  # Adjust this threshold as needed
new_node_id_prefix = "additional_"
new_node_counter = 0
additional_nodes = []

for i, j, w in zip(mst.row, mst.col, mst.data):
    if w > introduce_threshold:
        midpoint = ((tower_nodes[i][1][0] + tower_nodes[j][1][0]) / 2,
                    (tower_nodes[i][1][1] + tower_nodes[j][1][1]) / 2)
        new_node_id = new_node_id_prefix + str(new_node_counter)
        additional_nodes.append((new_node_id, midpoint))
        new_node_counter += 1

# Add new nodes to the graph and list of positions
for node_id, coords in additional_nodes:
    G.add_node(node_id, pos=coords)
    tower_nodes.append((node_id, coords))
    positions.append(coords)

# Add edges from MST to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    G.add_edge(node_i, node_j, weight=w)

# Cluster with DBSCAN and add new nodes
db = DBSCAN(eps=0.5, min_samples=2).fit(positions)
labels = db.labels_

# Generate new nodes within dense regions (clusters)
cluster_nodes = []

for label in set(labels):
    if label != -1:
        members = [positions[idx] for idx, _ in enumerate(tower_nodes) if labels[idx] == label]
        centroid = (sum(x[0] for x in members) / len(members), sum(x[1] for x in members) / len(members))
        cluster_nodes.append(centroid)

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes and new nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add edges from MST to the map
for edge in G.edges():
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add cluster nodes to the map
for node in cluster_nodes:
    folium.Marker(location=node, icon=folium.Icon(icon='star')).add_to(m)

m


In [26]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix, KDTree
from scipy.sparse.csgraph import minimum_spanning_tree

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Create a NetworkX graph
G = nx.Graph()

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Add nodes to the graph with 'pos' attribute
for node_id, coords in tower_nodes:
    G.add_node(node_id, pos=coords)

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    G.add_edge(node_i, node_j, weight=w)

# Use KNN for additional connections
kdtree = KDTree(positions)
for idx, position in enumerate(positions):
    distances, indices = kdtree.query([position], k=4)  # k=4 includes the point itself
    for neighbor_idx in indices[0][1:]:  # skip the first index (self)
        if len(G.adj[tower_nodes[idx][0]]) < 3:  # ensure no more than 3 connections
            G.add_edge(tower_nodes[idx][0], tower_nodes[neighbor_idx][0])

# Further add connections based on distance threshold, ensuring no more than 3 connections per node
threshold = 1.5
for i in range(len(positions)):
    for j in range(i+1, len(positions)):
        if len(G.adj[tower_nodes[i][0]]) < 3 and len(G.adj[tower_nodes[j][0]]) < 3 and dist_matrix[i][j] < threshold:
            G.add_edge(tower_nodes[i][0], tower_nodes[j][0])

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add MST edges to the map
mst_edges = [(i, j) for i, j, w in zip(mst.row, mst.col, mst.data)]
for edge in mst_edges:
    start, end = edge
    locs = [G.nodes[tower_nodes[start][0]]['pos'], G.nodes[tower_nodes[end][0]]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add KNN and threshold edges to the map
other_edges = [edge for edge in G.edges() if edge not in mst_edges]
for edge in other_edges:
    start, end = edge
    locs = [G.nodes[start]['pos'], G.nodes[end]['pos']]
    folium.PolyLine(locs, color="green", weight=2.5, opacity=1).add_to(m)

m


In [28]:
import geopandas as gpd
import networkx as nx
import folium
from scipy.spatial import distance_matrix, KDTree
from scipy.sparse.csgraph import minimum_spanning_tree

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Initialize graph for MST
G_mst = nx.Graph()
for node_id, coords in tower_nodes:
    G_mst.add_node(node_id, pos=coords)

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

# Calculate Minimum Spanning Tree
mst = minimum_spanning_tree(dist_matrix).tocoo()

# Add edges from MST to the graph
for i, j, w in zip(mst.row, mst.col, mst.data):
    node_i, node_j = tower_nodes[i][0], tower_nodes[j][0]
    G_mst.add_edge(node_i, node_j, weight=w)

# Initialize graph for KNN
G_knn = nx.Graph()
for node_id, coords in tower_nodes:
    G_knn.add_node(node_id, pos=coords)

# Use KNN for connections
kdtree = KDTree(positions)
for idx, position in enumerate(positions):
    distances, indices = kdtree.query([position], k=4)  # k=4 includes the point itself
    for neighbor_idx in indices[0][1:]:  # skip the first index (self)
        G_knn.add_edge(tower_nodes[idx][0], tower_nodes[neighbor_idx][0])

# Initialize graph for threshold-based connections
G_thresh = nx.Graph()
for node_id, coords in tower_nodes:
    G_thresh.add_node(node_id, pos=coords)

# Add connections based on distance threshold
threshold = 1.5
for i in range(len(positions)):
    for j in range(i+1, len(positions)):
        if dist_matrix[i][j] < threshold:
            G_thresh.add_edge(tower_nodes[i][0], tower_nodes[j][0])

# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m)

# Add MST edges to the map
for edge in G_mst.edges():
    start, end = edge
    locs = [G_mst.nodes[start]['pos'], G_mst.nodes[end]['pos']]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1).add_to(m)

# Add KNN edges to the map
for edge in G_knn.edges():
    start, end = edge
    locs = [G_knn.nodes[start]['pos'], G_knn.nodes[end]['pos']]
    folium.PolyLine(locs, color="green", weight=2.5, opacity=1).add_to(m)
m
